In [ ]:
%%capture
!pip install python-dotenv langchain langchain-openai langchain-core langgraph langchain-anthropic langchainhub langchain-community langchain-core arxiv datasets llama-parse langchain-experimental transformers langchain-huggingface torch peft trl bitsandbytes accelerate openai qdrant-client langchain-cohere langgraph pip install google-search-results -qU

In [7]:
from operator import itemgetter
from time import time
from typing import Any, Dict, TypedDict, List, Annotated, Sequence, Union
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeColors
from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import END, StateGraph
from nest_asyncio import apply as nest_asyncio_apply
import nest_asyncio
from IPython.display import Markdown
from langchain.prompts import SystemMessagePromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import Qdrant
from langchain_openai.embeddings import OpenAIEmbeddings
from qdrant_client import AsyncQdrantClient, QdrantClient, models
from tqdm import tqdm
import argparse
import asyncio
import os
from langchain_text_splitters import CharacterTextSplitter
from datasets import Dataset
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from llama_parse import LlamaParse
from pathlib import Path

In [8]:
nest_asyncio_apply()

In [28]:
MODEL_DIMENSIONS = {
    "text-embedding-3-small": 1536,
    "text-embedding-3-large": 3072,
}

url = ""
key = ""

In [34]:
import openai
openai.api_key = ""
from openai import OpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key="",temperature=0, streaming=True)
EMBEDDING_MODEL = "text-embedding-3-large"
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key="")
client = QdrantClient(url=url, api_key=key, prefer_grpc=True)
aclient = AsyncQdrantClient(url=url, api_key=key, prefer_grpc=True)

In [35]:
qdrant_args = {
    "client": client,
    "async_client": aclient,
    "embeddings": embeddings,
}

In [36]:
DATA = Path("data")

In [19]:
# Parse documents using LlamaParse
parser = LlamaParse(
    api_key='',
    result_type="markdown",
    verbose=True,
    language="en"
)
documents = await parser.aload_data(file_path=str(DATA / f"AWS.pdf"))

Started parsing the file under job_id 0276cfc2-69c4-4508-a8ff-87429fa8ed44


In [20]:
print(len(documents))

1


In [21]:
async def prepare_documents(documents):
    """
    Split documents into chunks.
    Args:
        documents (List[Document]): List of Document objects.
    """
    text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings(model="text-embedding-3-large", api_key=""))
    splitted_documents = await text_splitter.atransform_documents(documents=[Document(page_content=doc.text) for doc in documents])

    #splitted_docs = text_splitter.create_documents([Document(page_content=doc.text) for doc in documents])
    return splitted_documents

In [22]:
first = await prepare_documents(documents)
print(first)

[Document(page_content='# AWS Well-Architected FrameworkJuly 2019\n\nThis document describes the AWS Well-Architected Framework, which enables you to review and improve your cloud-based architectures and better understand the business impact of your design decisions. We address general design principles as well as specific best practices and guidance in five conceptual areas that we define as the pillars of the Well-Architected Framework. ---\n# AWS Well-Architected Framework\n\nNotices\n\nCustomers are responsible for making their own independent assessment of the information in this document. This document: (a) is for informational purposes only, (b) represents current AWS product offerings and practices, which are subject to change without notice, and (c) does not create any commitments or assurances from AWS and its affiliates, suppliers or licensors. AWS products or services are provided “as is” without warranties, representations, or conditions of any kind, whether express or imp

In [23]:
print(len(first))

62


In [29]:
async def main(documents, collection_name: str, embedding_model: str) -> None:
    """
    Main function to process documents and upload them to Qdrant.

    Args:
        collection_name (str): Name of the collection in Qdrant.
        embedding_model (str): Embedding model to use for document vectors.
    """
    client = AsyncQdrantClient(url=url, api_key=key, prefer_grpc=True)


    await client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=MODEL_DIMENSIONS[embedding_model], distance=models.Distance.COSINE),
    )


    documents = await prepare_documents(documents)
    embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key="")

    vector_store = Qdrant(
        client=QdrantClient(url=url, api_key=key, prefer_grpc=True),
        async_client=client,
        collection_name=collection_name,
        embeddings=embeddings,
    )

    await vector_store.aadd_documents(documents=documents)


In [30]:
asyncio.run(
    main(
        documents,
        collection_name="Reflection-Agent-AWS",
        embedding_model="text-embedding-3-large",
    )
)

<ipython-input-29-17707a731584>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  await client.recreate_collection(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [31]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [37]:
constitucion_vectorstore = Qdrant(collection_name="Reflection-Agent-AWS", **qdrant_args)
constitucion_retriever = constitucion_vectorstore.as_retriever(search_kwargs ={"k": 3})
pretty_print_docs(await constitucion_retriever.ainvoke("What is AWS Framework?"))

Document 1:

# AWS Well-Architected FrameworkJuly 2019

This document describes the AWS Well-Architected Framework, which enables you to review and improve your cloud-based architectures and better understand the business impact of your design decisions. We address general design principles as well as specific best practices and guidance in five conceptual areas that we define as the pillars of the Well-Architected Framework. ---
# AWS Well-Architected Framework

Notices

Customers are responsible for making their own independent assessment of the information in this document. This document: (a) is for informational purposes only, (b) represents current AWS product offerings and practices, which are subject to change without notice, and (c) does not create any commitments or assurances from AWS and its affiliates, suppliers or licensors. AWS products or services are provided “as is” without warranties, representations, or conditions of any kind, whether express or implied. The responsi

In [38]:
collections = await aclient.get_collections()
collection_names = [c["name"] for c in collections.dict()["collections"]]
collection_names

['Reflection-Agent-AWS',
 'Medical-Rag',
 'RAG-ANADOLU',
 'collection_name',
 'collection_namee']

In [39]:
retrievers = []
for collection in tqdm(collection_names, total=len(collection_names)):
    vectorstore = Qdrant(collection_name=collection, **qdrant_args)
    retriever = vectorstore.as_retriever(search_kwargs ={"k": 20})
    retrievers.append(retriever)

100%|██████████| 5/5 [00:00<00:00, 11250.82it/s]


In [40]:
collection_retriever_map = dict(zip(collection_names, retrievers))
collection_retriever_map

{'Reflection-Agent-AWS': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b880>, search_kwargs={'k': 20}),
 'Medical-Rag': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b8e0>, search_kwargs={'k': 20}),
 'RAG-ANADOLU': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b910>, search_kwargs={'k': 20}),
 'collection_name': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b850>, search_kwargs={'k': 20}),
 'collection_namee': VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b8b0>, search_kwargs={'k': 20})}

In [41]:
from langchain_cohere import CohereRerank

cohere_rerank = CohereRerank(cohere_api_key = "", top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank,
    base_retriever=collection_retriever_map.get("Reflection-Agent-AWS")
)

In [42]:
pretty_print_docs(docs=compression_retriever.get_relevant_documents(query="What is AWS Framework?"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document 1:

Copyright © 2019 Amazon Web Services, Inc. or its affiliates
---
# Introduction

Definitions

On Architecture

General Design Principles

# The Five Pillars of the Framework

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization

# The Review Process

# Conclusion

# Contributors

# Further Reading

# Document Revisions

# Appendix: Questions, Answers, and Best Practices

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization
---
# AWS Well-Architected Framework

Introduction

The AWS Well-Architected Framework helps you understand the pros and cons
of decisions you make while building systems on AWS. By using the Framework
you will learn architectural best practices for designing and operating reliable,
secure, efficient, and cost-effective systems in the cloud. It provides a way for you to
consistently measure your architectures against best practices and identify areas for
improvement. The process for reviewin

In [43]:
cohere_rerank = CohereRerank(cohere_api_key = "", top_n=5)

collection_retriever_reranker_map = {}
for collection in tqdm(collection_retriever_map.keys(), total=len(collection_retriever_map.keys())):
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=cohere_rerank,
        base_retriever=collection_retriever_map.get(collection)
    )
    collection_retriever_reranker_map[collection] = compression_retriever

100%|██████████| 5/5 [00:00<00:00, 6078.70it/s]


In [44]:
collection_retriever_reranker_map

{'Reflection-Agent-AWS': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x78ed7ab340d0>, top_n=5, model='rerank-english-v3.0', cohere_api_key='bRB6hZk9VLEyNBAzTVRCDnEBCbkqnWAhbPmhEprP', user_agent='langchain:partner'), base_retriever=VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b880>, search_kwargs={'k': 20})),
 'Medical-Rag': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x78ed7ab340d0>, top_n=5, model='rerank-english-v3.0', cohere_api_key='bRB6hZk9VLEyNBAzTVRCDnEBCbkqnWAhbPmhEprP', user_agent='langchain:partner'), base_retriever=VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b8e0>, search_kwargs={'k': 20})),
 'RAG-ANADOLU': ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.cl

In [45]:
medical_description = "It is useful when faced with concepts or questions involving AWS Framework."

In [46]:
name_description_retriever_map = {
    "influenza_virus": {
        "retriever": collection_retriever_reranker_map.get("Reflection-Agent-AWS"),
        "name": "influenza_virus",
        "description": medical_description
    },
    "RAG-ANADOLU": {
        "retriever": collection_retriever_reranker_map.get("Medical-Rag"),
        "name": "insurance_anadolu_sigorta",
        "description": "Useful for insurance related questions."
    }
}

In [47]:
tools = []
for name in name_description_retriever_map:
    tool = create_retriever_tool(
        retriever=name_description_retriever_map.get(name).get("retriever"),
        name=name_description_retriever_map.get(name).get("name"),
        description=name_description_retriever_map.get(name).get("description")
    )
    tools.append(tool)

In [48]:
tools

[Tool(name='influenza_virus', description='It is useful when faced with concepts or questions involving AWS Framework.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x78edbbee6440>, retriever=ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x78ed7ab340d0>, top_n=5, model='rerank-english-v3.0', cohere_api_key='bRB6hZk9VLEyNBAzTVRCDnEBCbkqnWAhbPmhEprP', user_agent='langchain:partner'), base_retriever=VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x78ed82e0b880>, search_kwargs={'k': 20})), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x78edbbee65f0>, retriever=ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Clie

In [49]:
import os

os.environ["TAVILY_API_KEY"] = ""

In [50]:
google_scholar = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper(serp_api_key="", top_k_results=10))
arxiv = ArxivAPIWrapper(top_k_results=10)
tavily = TavilySearchResults(max_results=10)

In [51]:
print(tavily.invoke("Rocket Science"))

[{'url': 'https://www.nasa.gov/stem-content/beginners-guide-to-rockets/', 'content': 'News & Events\nMultimedia\nSuggested Searches\nFeatured\nNASA Tests New Spacecraft Propellant Gauge on Lunar Lander\nNASA’s Laser Navigation Tech Enables Commercial Lunar Exploration\nTiny NASA Cameras to Picture Interaction Between Lander, Moon’s Surface\nMissions\nHumans in Space\nEarth & Climate\nThe Solar System\nThe Universe\nScience\nAeronautics\nTechnology\nLearning Resources\nAbout NASA\nEspañol\nNews & Events\nMultimedia\nHighlights\nThird NASA Enabled Private Flight to Space Station Completes Safely\nNASA Expedition 71 Astronauts to Conduct Research aboard Space Station\nHubble Spots a Galaxy Shrouded by Stars\nHighlights\nThird NASA Enabled Private Flight to Space Station Completes Safely\nNASA Expedition 71 Astronauts to Conduct Research aboard Space Station\n NASA’s Hubble Traces ‘String of Pearls’ Star Clusters in Galaxy Collisions\nHighlights\nSpecial Report: Aeronautics Honors Ingenuit

In [52]:
class ReflectionState(TypedDict):
    keys: Dict[str, Any]
    iteration: int

In [53]:
class Feedback(BaseModel):
    """Represents feedback to improve response."""
    has_suggestions: bool = Field(..., description="Indicates whether there are suggestions for further improvements. True if there are suggestions, False otherwise.")
    suggestions: str = Field(..., description="Feedback and suggestions for improving the response. If there are no suggestions, this should be an empty string.")

In [54]:
generator_system_prompt = """You are an AI research assistant with expertise in conducting literature reviews. Your role is to help the user thoroughly survey the scholarly literature on a particular research topic or question.

Remember, your goal is to save the user significant time and effort by finding, filtering, and making sense of large bodies of academic literature and documents. Using documents and academic papers, provide them with a comprehensive, well-organized, referenced, and insightful overview that they can build upon for their own research or use to get quickly up to speed on the topic.

Only return your answer.
"""


user_prompt = """Write a literature review on the topic about '{question}' using only the following research informations:

###Research papers:
<academic_papers>
{research}
</academic_papers>


###Medical documents:
<medical_documents>
{documents}
</medical_documents>
"""
reviewer_system_prompt = """You are an AI response reviewer assistant specializing in analyzing and providing feedback on response about relevant topics in healthcare field. You are provided with research papers, documents and an answer based on them. Carefully review the given answer and analyze its accuracy. Assess potential improvements to make the response accurate, complete, and detailed, also clearly understandable to the researcher. Follow these guidelines:

-If you identify areas for improvement, give specific actionable suggestions on how the response could be refactored, simplified, or enhanced. Be as clear and concrete as possible

-If the answer is sufficiently detailed, accurate and clear, simply return an empty string to indicate no relevant feedback.

Carefully analyze the given response and provide a thoughtful evaluation covering the aspects mentioned above. If you have suggestions, return them as a single text block. If there is no useful feedback to provide, respond with an empty string."""

prompt_general = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a helpfull assistant"),
        ("user", user_prompt)
    ]
)

In [55]:
fixer_system_prompt = """You are a helper AI assistant that takes responses and feedback about it, and also takes research papers and documents about related to response, then provides improved, and detailed versions of the response based on that feedback, papers and documents. Your goal is to incorporate the suggested improvements and generate an optimized version of the response. Follow these guidelines:

- Carefully apply the changes suggested by the feedback to the response.
- Ensure that the improved response is fully executable, free of incorrect or irrelevant information, and incorporates all the suggested enhancements.
- Return only the improved response.

Your entire response should consist solely of the improved response, focusing on providing the most refined and efficient based on the given feedback."""

In [56]:
print(prompt_general)

input_variables=['documents', 'question', 'research'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpfull assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['documents', 'question', 'research'], template="Write a literature review on the topic about '{question}' using only the following research informations:\n\n###Research papers:\n<academic_papers>\n{research}\n</academic_papers>\n\n\n###Medical documents:\n<medical_documents>\n{documents}\n</medical_documents>\n"))]


In [57]:
def organize_research(findings: Dict[str, Any]) -> str:
    return (
        findings["academic2"] + "\n\n"
        + "\n\n".join([str(result["content"]) for result in findings["tavily"]])
        + findings["academic1"] + "\n\n"
    )

In [58]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [59]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [60]:
def exe_agent(input):
  agent = create_openai_tools_agent(llm, tools, prompt)
  agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)
  response=agent_executor.invoke({"input": input})
  return response['intermediate_steps'][0][-1]

In [61]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)
response=agent_executor.invoke({"input": "What is AWS Framework?"})
print(response['intermediate_steps'][0][-1])



> Entering new AgentExecutor chain...

Invoking: `influenza_virus` with `{'query': 'What is AWS Framework?'}`


Copyright © 2019 Amazon Web Services, Inc. or its affiliates
---
# Introduction

Definitions

On Architecture

General Design Principles

# The Five Pillars of the Framework

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization

# The Review Process

# Conclusion

# Contributors

# Further Reading

# Document Revisions

# Appendix: Questions, Answers, and Best Practices

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization
---
# AWS Well-Architected Framework

Introduction

The AWS Well-Architected Framework helps you understand the pros and cons
of decisions you make while building systems on AWS. By using the Framework
you will learn architectural best practices for designing and operating reliable,
secure, efficient, and cost-effective systems in the cloud. It provides a way for you to
consistently measure y

In [62]:
rag_chain = (
    RunnableParallel(academic1=google_scholar, academic2=RunnableLambda(func=lambda x: arxiv.run(x), name="Arxiv"), tavily=tavily, documents=RunnableLambda(func=exe_agent, name="get_docs"), question=RunnablePassthrough())
    | {"research": RunnableLambda(func=organize_research, name="organize_research"), "question": itemgetter("question"), "documents": itemgetter("documents")}
    | prompt_general
)

In [63]:
topic = "What is AWS Framework?"
start = time()
stream = rag_chain.stream(input=topic)
for token in stream:
    print(token, end="", flush=True)



> Entering new AgentExecutor chain...

Invoking: `influenza_virus` with `{'query': 'What is AWS Framework?'}`


Copyright © 2019 Amazon Web Services, Inc. or its affiliates
---
# Introduction

Definitions

On Architecture

General Design Principles

# The Five Pillars of the Framework

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization

# The Review Process

# Conclusion

# Contributors

# Further Reading

# Document Revisions

# Appendix: Questions, Answers, and Best Practices

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization
---
# AWS Well-Architected Framework

Introduction

The AWS Well-Architected Framework helps you understand the pros and cons
of decisions you make while building systems on AWS. By using the Framework
you will learn architectural best practices for designing and operating reliable,
secure, efficient, and cost-effective systems in the cloud. It provides a way for you to
consistently measure y

In [64]:
topic = "What is AWS Framework?"
responsee = rag_chain.invoke(input=topic)



> Entering new AgentExecutor chain...

Invoking: `influenza_virus` with `{'query': 'What is AWS Framework?'}`


Copyright © 2019 Amazon Web Services, Inc. or its affiliates
---
# Introduction

Definitions

On Architecture

General Design Principles

# The Five Pillars of the Framework

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization

# The Review Process

# Conclusion

# Contributors

# Further Reading

# Document Revisions

# Appendix: Questions, Answers, and Best Practices

Operational Excellence
Security
Reliability
Performance Efficiency
Cost Optimization
---
# AWS Well-Architected Framework

Introduction

The AWS Well-Architected Framework helps you understand the pros and cons
of decisions you make while building systems on AWS. By using the Framework
you will learn architectural best practices for designing and operating reliable,
secure, efficient, and cost-effective systems in the cloud. It provides a way for you to
consistently measure y

In [65]:
async def get_relevant_informations(state):
  rag_chain = (
      RunnableParallel(academic1=google_scholar, academic2=RunnableLambda(func=lambda x: arxiv.run(x), name="Arxiv"), tavily=tavily, documents=RunnableLambda(func=exe_agent, name="get_docs"), question=RunnablePassthrough())
      | {"research": RunnableLambda(func=organize_research, name="organize_research"), "question": itemgetter("question"), "documents": itemgetter("documents")}
      | prompt_general
  )
  actual_prompt = await rag_chain.ainvoke(input=state["keys"]["question"])
  return {"keys": {"question": state["keys"]["question"], "actual_prompt": actual_prompt.messages[1].content}}

In [66]:
# Code generator node definition
async def generate_response(state):
    prompt_generate = ChatPromptTemplate.from_messages(
        messages=[
            ("system", generator_system_prompt),
            ("user", "<topic> {topic} </topic>"+ "\n\n" + state["keys"]["actual_prompt"])
        ]
    )
    runnable = prompt_generate | llm | StrOutputParser()
    print(type(state["keys"]["actual_prompt"]))
    #response = await runnable.ainvoke(input={"question": state["keys"]["question"]})
    response = await runnable.ainvoke({"topic": "AWS Framework"})
    return {"keys": {"question": state["keys"]["question"], "actual_prompt": state["keys"]["actual_prompt"], "response": response}, "iteration": 0}

In [67]:
# Code reviewer node definition
async def review_response(state):
    prompt_review = ChatPromptTemplate.from_messages(
        messages=[
            ("system", reviewer_system_prompt),
            ("user", state["keys"]["actual_prompt"] + "\n\n" + "<response> {response} </response>")
        ]
    )
    runnable = prompt_review | llm.with_structured_output(Feedback)
    feedback = await runnable.ainvoke(input={"response": state["keys"]["response"]})

    return {"keys": {**state["keys"], "feedback": feedback}, "iteration": state["iteration"]}

In [68]:
# Code fixer node definition
async def fix_response(state):
    prompt_fix = ChatPromptTemplate.from_messages(
        messages=[
            ("system", fixer_system_prompt),
            ("user", state["keys"]["actual_prompt"] + "\n\n <response> {response} </response>\n\n <feedback> {feedback} </feedback>")
        ]
    )
    runnable = prompt_fix | llm | StrOutputParser()
    improved_response = await runnable.ainvoke(input={
        "response": state["keys"]["response"],
        "feedback": state["keys"]["feedback"].suggestions
    })
    return {"keys": {"question": state["keys"]["question"], "actual_prompt": state["keys"]["actual_prompt"], "response": improved_response}, "iteration": state["iteration"] + 1}

In [69]:
# Decide when to keep improving the code and when to stop
def should_fix_response(state):
    if any([state["iteration"] >= 5, not state["keys"]["feedback"].has_suggestions]):
        return "end"
    return "improve"

In [70]:
# Define workflow
workfloww = StateGraph(ReflectionState)

# Add nodes
workfloww.add_node("prompt_generator", get_relevant_informations)
workfloww.add_node("response_generator", generate_response)
workfloww.add_node("response_reviewer", review_response)
workfloww.add_node("response_fixer", fix_response)

# Add edges
workfloww.add_edge(start_key="prompt_generator", end_key="response_generator")
workfloww.add_edge(start_key="response_generator", end_key="response_reviewer")
workfloww.add_edge(start_key="response_fixer", end_key="response_reviewer")

# Add conditional edges
workfloww.add_conditional_edges(
    "response_reviewer",
    should_fix_response,
    {
        "improve": "response_fixer",
        "end": END
    }
)

# Set entry point
workfloww.set_entry_point("prompt_generator")

In [71]:
app = workfloww.compile(debug=True)

In [72]:
out = RunnableLambda(name="Answer Extraction", func=lambda state: state["keys"]["response"])
runnable = (app | out)

In [73]:
_question = "What is AWS Framework?"
_input = {
    "input": {"keys": {"question": _question}}
}
response = await runnable.ainvoke(**_input)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
                  'Conclusion\n'
                  '\n'
                  'The AWS Well-Architected Framework provides architectural '
                  'best practices across the five pillars for designing and '
                  'operating reliable, secure, efficient, and cost-effective '
                  'systems in the cloud. The Framework provides a set of '
                  'questions that allows you to review an existing or proposed '
                  'architecture. It also provides a set of AWS best practices '
                  'for each pillar. Using the Framework in your architecture '
                  'will help you produce stable and efficient systems, which '
                  'allow you to focus on your functional requirements. 41\n'
                  '---\n'
                  '# AWS Well-Architected Framework\n'
                  '\n'
                  '# Contributors\n'
                  '\n'
      